## Preprocessing Footytables Data

In this notebook we preprocess the data collected from footy tables as it contains the brownlow votes from 2022.

The aim is to derive the columns of interest and merge it with our original dataframe so that we have the brownlow votes information for all seasons.

Unfortunately there is no nice key to merge the dataframes on and when merging via the first and last name of players, some the result was not as expected. This was because of the represenation of some names that included, two words like "De Goey" or "De Koning", names with apostrophes like "O'hallaron" and names with an initial like "Josh P. Kennedy". There is also the case where names such as "Thomas" were shortened to "Tom"

So the names had to transformed to the same format so that they could be merged as intended.

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/landing/player_stats_22_tables.csv')
full_df = pd.read_parquet('../../data/raw/cleaned_stats_12-22.parquet')

In [3]:
df.columns

Index(['Unnamed: 0', 'Season', 'Round', 'Date', 'Local.start.time', 'Venue',
       'Attendance', 'Home.team', 'HQ1G', 'HQ1B', 'HQ2G', 'HQ2B', 'HQ3G',
       'HQ3B', 'HQ4G', 'HQ4B', 'Home.score', 'Away.team', 'AQ1G', 'AQ1B',
       'AQ2G', 'AQ2B', 'AQ3G', 'AQ3B', 'AQ4G', 'AQ4B', 'Away.score',
       'First.name', 'Surname', 'ID', 'Jumper.No.', 'Playing.for', 'Kicks',
       'Marks', 'Handballs', 'Goals', 'Behinds', 'Hit.Outs', 'Tackles',
       'Rebounds', 'Inside.50s', 'Clearances', 'Clangers', 'Frees.For',
       'Frees.Against', 'Brownlow.Votes', 'Contested.Possessions',
       'Uncontested.Possessions', 'Contested.Marks', 'Marks.Inside.50',
       'One.Percenters', 'Bounces', 'Goal.Assists', 'Time.on.Ground..',
       'Substitute', 'Umpire.1', 'Umpire.2', 'Umpire.3', 'Umpire.4',
       'group_id'],
      dtype='object')

In [4]:
df

,Unnamed: 0,Season,Round,Date,Local.start.time,Venue,Attendance,Home.team,HQ1G,HQ1B,...,One.Percenters,Bounces,Goal.Assists,Time.on.Ground..,Substitute,Umpire.1,Umpire.2,Umpire.3,Umpire.4,group_id
0,1,2022,1,2022-03-16,1910,M.C.G.,58002,Melbourne,4,5,...,2,1,0,55,NaN,John Howorth,Rob Findlay,Jacob Mollison,NaN,NaN
1,2,2022,1,2022-03-16,1910,M.C.G.,58002,Melbourne,4,5,...,2,0,0,73,NaN,John Howorth,Rob Findlay,Jacob Mollison,NaN,NaN
2,3,2022,1,2022-03-16,1910,M.C.G.,58002,Melbourne,4,5,...,1,0,0,83,NaN,John Howorth,Rob Findlay,Jacob Mollison,NaN,NaN
3,4,2022,1,2022-03-16,1910,M.C.G.,58002,Melbourne,4,5,...,2,0,0,86,NaN,John Howorth,Rob Findlay,Jacob Mollison,NaN,NaN
4,5,2022,1,2022-03-16,1910,M.C.G.,58002,Melbourne,4,5,...,0,0,1,81,NaN,John Howorth,Rob Findlay,Jacob Mollison,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9517,9518,2022,GF,2022-09-24,1430,M.C.G.,100024,Geelong,6,5,...,6,1,0,90,NaN,Simon Meredith,Matt Stevic,Brendan Hosking,NaN,NaN
9518,9519,2022,GF,2022-09-24,1430,M.C.G.,100024,Geelong,6,5,...,1,0,0,38,NaN,Simon Meredith,Matt Stevic,Brendan Hosking,NaN,NaN
9519,9520,2022,GF,2022-09-24,1430,M.C.G.,100024,Geelong,6,5,...,4,0,1,72,NaN,Simon Meredith,Matt Stevic,Brendan Hosking,NaN,NaN
9520,9521,2022,GF,2022-09-24,1430,M.C.G.,100024,Geelong,6,5,...,0,3,1,78,NaN,Simon Meredith,Matt Stevic,Brendan Hosking,NaN,NaN


In [5]:
rounds = [str(i) for i in range(1,24)]
df = df.query('Round.isin(@rounds)')

In [6]:
keep_col = [
    'Home.team', 'Away.team', 'Round', 'First.name', 'Surname', 'Playing.for', 'Brownlow.Votes'
]

In [7]:
df = df[keep_col]

In [8]:
for col in keep_col:
    print(f'"{col}": "{col.lower().replace(".","_")}",')

"Home.team": "home_team",
"Away.team": "away_team",
"Round": "round",
"First.name": "first_name",
"Surname": "surname",
"Playing.for": "playing_for",
"Brownlow.Votes": "brownlow_votes",


In [9]:
df.rename(columns={
    "Home.team": "home_team",
    "Away.team": "away_team",
    "Round": "round_number",
    "First.name": "first_name",
    "Surname": "surname",
    "Playing.for": "playing_for",
    "Brownlow.Votes": "brownlow_votes"
}, inplace=True)

In [10]:
df = df.astype({
    'round_number':'int'
})

In [11]:
df

,home_team,away_team,round_number,first_name,surname,playing_for,brownlow_votes
0,Melbourne,Western Bulldogs,1,Toby,Bedford,Melbourne,0
1,Melbourne,Western Bulldogs,1,Jake,Bowey,Melbourne,0
2,Melbourne,Western Bulldogs,1,Angus,Brayshaw,Melbourne,0
3,Melbourne,Western Bulldogs,1,Ben,Brown,Melbourne,0
4,Melbourne,Western Bulldogs,1,Bayley,Fritsch,Melbourne,0
...,...,...,...,...,...,...,...
9103,St Kilda,Sydney,23,Sam,Reid,Sydney,0
9104,St Kilda,Sydney,23,James,Rowbottom,Sydney,0
9105,St Kilda,Sydney,23,Dylan,Stephens,Sydney,0
9106,St Kilda,Sydney,23,Chad,Warner,Sydney,0


In [12]:
# realistically we only need to merge rows where a player polled, and all remaining will be na
# which can be filled with .fillna(0)
polled = df.query('brownlow_votes > 0')

In [13]:
polled.head()

,home_team,away_team,round_number,first_name,surname,playing_for,brownlow_votes
14,Melbourne,Western Bulldogs,1,Clayton,Oliver,Melbourne,1
15,Melbourne,Western Bulldogs,1,Christian,Petracca,Melbourne,3
35,Melbourne,Western Bulldogs,1,Jack,Macrae,Western Bulldogs,2
46,Carlton,Richmond,1,Adam,Cerra,Carlton,3
47,Carlton,Richmond,1,Patrick,Cripps,Carlton,1


In [14]:
# 3 players * 9 games * 22 rounds (exclude bye round)
len(polled)

594

In [15]:
df_2022 = full_df.loc[full_df['season'] == 2022]

In [16]:
df_2022

,match_id,match_home_team,match_away_team,match_date,match_round,season,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,...,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils
85008,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,0,1,23,4,0,0,0,0
85009,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,3,0,3,9,0,1,0,10
85010,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,8,64,1,0,15,10,53,5,2,3
85011,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,1,0,16,6,0,1,1,4
85012,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,1,0,4,2,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94111,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,0,0,0,1,8,5,0,1,3,0
94112,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,0,0,1,0,21,6,0,1,1,0
94113,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,0,0,0,0,23,5,0,0,2,1
94114,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,0,0,0,3,13,2,0,2,2,0


In [17]:
cols_2022 = ['match_round', 'player_first_name', 'player_last_name', 'player_team']

In [18]:
# we get 8531 rows when we expected 8514 (9108 - 594)
# therefore there are 17 cases where the player's names are represented differently

new_df = pd.merge(df_2022, polled, 
         left_on=['player_first_name', 'player_last_name', 'match_round'], 
         right_on=['first_name', 'surname', 'round_number'],
        how='left')

# we get 8531 rows when we expected 8514 (9108 - 594)
# therefore there are 17 cases where the player's names are represented differently

len(new_df[new_df.isna().any(axis=1)])

8531

In [19]:
interim_df = pd.merge(df_2022, df, 
         left_on=['player_first_name', 'player_last_name', 'match_round'], 
         right_on=['first_name', 'surname', 'round_number'],
        how='left')
len(interim_df[interim_df.isna().any(axis=1)])

346

In [20]:
# list of last names where there is a naming difference
interim_df[interim_df.isna().any(axis=1)][cols_2022].player_last_name.unique()

array(['Williamson', "O'Brien", 'De Goey', 'Kennedy', 'de Boer',
       'Himmelberg', "O'Riordan", "O'Halloran", "O'Meara", 'Hinge',
       'Macpherson', 'Rioli', "O'Neill", 'Williams', 'Dewar',
       "O'Driscoll", 'Ryder', 'Taberner', "O'Connor", 'Brown', 'Petty',
       'Berry', 'Jones', "D'Ambrosio", 'Oea'], dtype=object)

In [21]:
# list of last names where there is a naming difference
polled.surname.unique()

array(['Oliver', 'Petracca', 'Macrae', 'Cerra', 'Cripps', 'Kennedy',
       'Hayes', 'de Goey', 'Lipinski', 'Dangerfield', 'Hawkins', 'Martin',
       'Green', 'Heeney', 'Parker', 'Neale', 'Boak', 'Houston', 'Jiath',
       'Mitchell', 'McDonald', 'Keays', 'Rachele', 'Brayshaw', 'Miller',
       'Rankine', 'Rowell', 'Curnow', 'McKay', 'Franklin', 'Mills',
       'Crisp', 'Merrett', 'McCluggage', 'Lewis', 'Greenwood', 'Larkey',
       'Ziebell', 'Balta', 'Short', 'Taranto', 'Higgins', 'King',
       'Dunkley', 'English', 'Smith', 'Langdon', 'Dawson', 'Marshall',
       'Kelly', 'Elliott', 'Cameron', 'Stewart', 'Lyons', 'McCarthy',
       'Zorko', 'Docherty', 'Hewett', 'Gresham', 'Steele', 'Brodie',
       'Schultz', 'Gawn', 'Harmes', 'McStay', 'McInerney', 'Simpkin',
       'Daicos', 'Grundy', 'McGovern', 'Mundy', 'Bolton', 'Prestia',
       'Parish', 'Walker', 'Hill', 'Sinclair', 'Anderson', 'Weller',
       'Williams', 'Bontempelli', 'Naughton', 'Ladhams', 'Walsh', 'Darcy',
       'Ta

In [22]:
# browsing both lists, these are the intersection

fix = [
    'Obrien', 'de Goey',  'Kennedy', 'Himmelberg', 'Omeara',  'rioli', 'Williams', 
    'ryder', 'taberner', 'brown', 'berry', 
]

We query each dataframe with the incorrect names, and overwrite the polled dataframe to match the 2022 dataframe.

Not ideal that the indexes are hardcoded, but due to the nature of the problem, each is unique and fairly specific since they are names that are being changed.

In [23]:
polled.query('surname == "Kennedy"')

,home_team,away_team,round_number,first_name,surname,playing_for,brownlow_votes
54,Carlton,Richmond,1,Matthew,Kennedy,Carlton,2
5485,West Coast,Essendon,15,Josh,Kennedy,West Coast,3
6612,West Coast,Carlton,17,Matthew,Kennedy,Carlton,2
8246,West Coast,Adelaide,21,Josh,Kennedy,West Coast,3


In [24]:
df_2022.query('player_last_name == "Kennedy"')[cols_2022][:5]

,match_round,player_first_name,player_last_name,player_team
85077,1,Matthew,Kennedy,Carlton
85193,1,Josh P.,Kennedy,Sydney
85377,1,Josh J.,Kennedy,West Coast
85492,2,Matthew,Kennedy,Carlton
85517,2,Josh P.,Kennedy,Sydney


In [25]:
polled.loc[2157, 'surname'] = "O'Brien"

In [26]:
polled.loc[[122, 512, 1686, 2467, 7506], 'surname'] = "De Goey"

In [27]:
polled.loc[[5485, 8246], 'first_name'] = 'Josh J.'

In [28]:
polled.loc[[5045, 5736], 'first_name'] = "Harry"

In [29]:
polled.loc[[4962, 7930], 'surname'] = "O'Meara"

In [30]:
polled.loc[5488, 'first_name'] = 'Junior'

In [31]:
# no fix for williams, name difference was for Bailey J. Williams, but player that polled was Zac Williams

In [32]:
polled.loc[[3465, 5880], 'first_name'] = 'Paddy'

In [33]:
polled.loc[2019, 'first_name'] = 'Matt'

In [34]:
polled.loc[6119, 'first_name'] = 'Callum M.'

In [35]:
# Sam and Jarrod polled, Thomas Berry shows as n/a, but will be set to zero

In [36]:
new_df = pd.merge(df_2022, polled, 
         left_on=['player_first_name', 'player_last_name', 'match_round'], 
         right_on=['first_name', 'surname', 'round_number'],
        how='left')

# successfully fixed all 17 incorrect cases

len(new_df[new_df.isna().any(axis=1)])

8514

In [37]:
# this tells us we wish to keep the y column
new_df[['player_first_name', 'player_last_name', 
        'match_id', 'brownlow_votes_x', 'brownlow_votes_y']].sort_values('brownlow_votes_y')

,player_first_name,player_last_name,match_id,brownlow_votes_x,brownlow_votes_y
9078,Jack,Steele,16346,0,1.0
2082,Josh,Kelly,16168,0,1.0
6968,Christian,Petracca,16293,0,1.0
3785,Ollie,Wines,16211,0,1.0
2144,Bailey,Smith,16169,0,1.0
...,...,...,...,...,...
9103,Logan,McDonald,16346,0,NaN
9104,Errol,Gulden,16346,0,NaN
9105,Mitch,Owens,16346,0,NaN
9106,Nasiah,Wanganeen-Milera,16346,0,NaN


In [38]:
# set all columns where brownlow votes is na to null
new_df.loc[new_df['brownlow_votes_y'].isna(), 'brownlow_votes_y'] = 0

In [39]:
new_df.rename(columns={'brownlow_votes_y': 'brownlow_votes'}, inplace=True)

In [40]:
new_df

,match_id,match_home_team,match_away_team,match_date,match_round,season,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,...,score_launches,shots_at_goal,spoils,home_team,away_team,round_number,first_name,surname,playing_for,brownlow_votes
0,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,1,0,10,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,5,2,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,1,1,4,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,16117,Melbourne,Western Bulldogs,2022-03-16,1,2022,14,13,97,11,...,0,0,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9103,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9104,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9105,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,0,2,1,NaN,NaN,NaN,NaN,NaN,NaN,0.0
9106,16346,St Kilda,Sydney,2022-08-21,23,2022,11,8,74,13,...,2,2,0,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [41]:
final_df_col = list(full_df.columns)

In [42]:
new_df = new_df[final_df_col]

In [43]:
# will be used with 2023 data to get past polling perforamnce
new_df.to_parquet('../../data/curated/brownlow_votes_22.parquet')

In [44]:
full_df = full_df.query('season < 2022')
full_df

,match_id,match_home_team,match_away_team,match_date,match_round,season,match_home_team_goals,match_home_team_behinds,match_home_team_score,match_away_team_goals,...,hitouts_to_advantage,hitout_win_percentage,intercept_marks,marks_on_lead,pressure_acts,rating_points,ruck_contests,score_launches,shots_at_goal,spoils
0,13960,Greater Western Sydney,Sydney,2012-03-24,1,2012,5,7,37,14,...,0,0,0,1,15,11,0,2,1,1
1,13960,Greater Western Sydney,Sydney,2012-03-24,1,2012,5,7,37,14,...,1,0,0,2,12,9,0,1,2,1
2,13960,Greater Western Sydney,Sydney,2012-03-24,1,2012,5,7,37,14,...,0,0,3,0,15,12,0,1,0,3
3,13960,Greater Western Sydney,Sydney,2012-03-24,1,2012,5,7,37,14,...,0,0,1,0,23,15,0,2,1,0
4,13960,Greater Western Sydney,Sydney,2012-03-24,1,2012,5,7,37,14,...,0,0,8,0,7,16,0,2,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85003,16105,Adelaide,North Melbourne,2021-08-22,23,2021,13,20,98,8,...,1,16,0,0,8,6,18,2,3,1
85004,16105,Adelaide,North Melbourne,2021-08-22,23,2021,13,20,98,8,...,0,0,0,1,15,3,0,0,1,0
85005,16105,Adelaide,North Melbourne,2021-08-22,23,2021,13,20,98,8,...,0,0,0,0,6,9,0,2,0,0
85006,16105,Adelaide,North Melbourne,2021-08-22,23,2021,13,20,98,8,...,0,0,0,0,9,4,0,0,3,0


In [45]:
final_df = pd.concat([full_df, new_df])

In [46]:
final_df.to_parquet('../../data/raw/cleaned_stats_12-22_fixed_bv.parquet')